In [ ]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
data = yf.download(ticker, period="1y", interval="1h")


In [ ]:
data

In [ ]:
# Darwinex Data
df = pd.read_csv('../Data/Darwinex/USDJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)


In [ ]:
# Importing through a CSV that has more data which was downloaded externally.
df = pd.read_csv('../Data/GBPJPY_H1(2008-01-25 - 2024-02-02).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
df.index = pd.to_datetime(df.index)


In [ ]:
df = df.iloc[:10000]

In [ ]:
df['ATR'] = ta.atr(pd.Series(df.High), 
                          pd.Series(df.Low), 
                          pd.Series(df.Close), 
                          length=14)

df['ATR']  = round(df['ATR'], 4)

In [ ]:
# csv slice by date
start_date = '2013-01-01'
end_date = '2023-01-01'
df = df.loc[start_date:end_date]

# Write out strategy here to figure out the logic

In [ ]:
class Strat(Strategy):
    def init(self):
        pass

    def next(self):
        pass

bt = Backtest(df_slice, Strat, cash=10_000)
bt.run()
bt.plot()

In [ ]:
class Strat(Strategy):
    r = 2
    def init(self):
        # Calculate the 50-period EMA and 14-period ATR and check engulfing        
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), length=50)
        self.atr = self.I(ta.atr, 
                          pd.Series(self.data.High), 
                          pd.Series(self.data.Low), 
                          pd.Series(self.data.Close), 
                          length=14)
               
        self.buy_pullback = False
        self.sell_pullback = False
        self.buy_pullback_count = 0
        self.sell_pullback_count = 0
        self.consolidation_high = float('inf')   
        self.consolidation_low = float('inf')  

        self.position_status = False

        self.custom_trades_log = []
        
    def next(self):
        
        current_ema = self.ema[-1]
        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]        
        current_low = self.data.Low[-1]        
        
        # Buy logic
        if crossover(current_close, current_ema) or (current_close < current_ema):
            self.buy_pullback = False
            self.buy_pullback_count = 0
            self.consolidation_high = -2

        # Pullback logic for long trades.
        if current_high > self.consolidation_high:
            self.consolidation_high = max(self.consolidation_high, current_high)
        if current_high < self.consolidation_high:
            if current_close < self.consolidation_high and current_close < current_open:
                self.buy_pullback_count += 1

            if self.buy_pullback_count >= 2:
                self.buy_pullback = True
            if not self.position:
                if current_close < self.consolidation_high and self.buy_pullback:
                    if current_close > self.data.High[-2]:
                        self.buy_pullback = False
                        self.buy_pullback_count = 0
                        self.consolidation_high = -2
                        sl = self.data.Low[-1] - self.atr[-1]
                        sl_pips = self.data.Close - sl
                        tp = self.data.Close[-1] + (sl_pips * self.r)
                        
                        # Log the initiation of a trade
                        self.custom_trades_log.append({
                            'entry_time': self.data.index[-1],
                            'entry_price': current_close,
                            'direction': 'BUY',
                            'sl': sl,
                            'tp': tp,
                            'exit_time': None,  # To be merged from backtesting _trades
                            'exit_price': None,  # To be merged from backtesting _trades
                        })
                        
                        # Execute the trade
                        self.buy(sl=sl, tp=tp)    
        
        
        # SELL logic
        if crossover(current_ema, current_close) or (current_close > current_ema):
            self.sell_pullback = False
            self.sell_pullback_count = 0
            self.consolidation_low = float('inf')

        # Pullback logic for long trades.
        if current_low < self.consolidation_low:
            self.consolidation_low = min(self.consolidation_low, current_low)
        if current_low > self.consolidation_low: # pullback logic condition start
            if current_close > self.consolidation_low and current_close > current_open:
                self.sell_pullback_count += 1

            if self.sell_pullback_count >= 2:
                self.sell_pullback = True
            if not self.position:
                if current_close > self.consolidation_low and self.sell_pullback:
                    if current_close < self.data.Low[-2]:
                        self.sell_pullback = False
                        self.sell_pullback_count = 0
                        self.consolidation_low = float('inf')
                        sl = self.data.High[-1] + self.atr[-1]
                        sl_pips = sl - self.data.Close[-1]
                        tp = self.data.Close[-1] - (sl_pips * self.r)
                        
                        # Log the initiation of a trade
                        self.custom_trades_log.append({
                            'entry_time': self.data.index[-1],
                            'entry_price': current_close,
                            'direction': 'SELL',
                            'sl': sl,
                            'tp': tp,
                            'exit_time': None,  # To be merged from backtesting _trades
                            'exit_price': None,  # To be merged from backtesting _trades
                        })
                        
                        # Execute the trade
                        self.sell(sl=sl, tp=tp)    

In [ ]:
bt = Backtest(df, Strat, cash=100000)
stats = bt.run()
print(stats)


In [ ]:
bt.optimize(r=[1, 1.2, 1.5, 2])

In [ ]:
bt.plot()


### Merge the _trades to my custom trade log to perform additional analysis

In [ ]:
# Creates the custom trade log into a df and merges the exit information from _trades to this custom df.
custom_df = pd.DataFrame(stats._strategy.custom_trades_log)
custom_df['exit_time'] = stats._trades['ExitTime']
custom_df['exit_price'] = stats._trades['ExitPrice']


In [ ]:
# Pips calculations
custom_df['stop_pips'] = round((custom_df['entry_price'] - custom_df['sl']) * 100, 2)
custom_df['result_pips'] = round((custom_df['exit_price'] - custom_df['entry_price']) * 100, 2)
custom_df['rr'] = round(custom_df['result_pips'] / custom_df['stop_pips'], 2)


In [ ]:
initial_equity = 100000.0
# Initialize account equity and risk for the first row only
custom_df.at[0, 'account_equity'] = initial_equity
custom_df.at[0, 'risk'] = initial_equity * 0.01
custom_df['pnl'] = 0.0  # Initialize pnl column

for index, row in custom_df.iterrows():
    # Calculate pnl from the first row
    if index > 0:
        # Use previous row's account equity to calculate risk for the current trade
        custom_df.at[index, 'risk'] = custom_df.at[index - 1, 'account_equity'] * 0.01
    
    # pnl calculation includes the first trade
    pnl = custom_df.at[index, 'risk'] * row['rr']
    custom_df.at[index, 'pnl'] = pnl
    
    
    if index == 0:
        custom_df.at[index, 'account_equity'] += pnl  
    else:
        custom_df.at[index, 'account_equity'] = custom_df.at[index - 1, 'account_equity'] + pnl

# Ensure data types
custom_df['account_equity'] = custom_df['account_equity'].astype(float)
custom_df['risk'] = custom_df['risk'].astype(float)
custom_df['pnl'] = custom_df['pnl'].astype(float)


In [ ]:
custom_df.head()

In [ ]:
total_trades = len(custom_df)
winning_trades = (custom_df['rr'] > 0).sum()
win_rate = f'{round((winning_trades / total_trades) * 100, 2)}%'
average_win_r = round(custom_df[custom_df['rr'] > 0]['rr'].mean(), 2)
net_pnl = custom_df['pnl'].sum()
closing_equity = round(custom_df['account_equity'].iloc[-1], 2)
percent_pnl = round((net_pnl / initial_equity) * 100, 2)

trade_info = {
    'Win Rate': win_rate,
    'Total Trades': total_trades,
    'Average Win Ratio': average_win_r,
    'Net PnL': net_pnl,
    'Closing Equity': f'${closing_equity}',
    'Return %': percent_pnl,
}

In [ ]:
for key, value in trade_info.items():
    print(f'{key}: {value}')

In [ ]:

# plot own equity curve based on backtesting data for data more than 10K
equity_curve = stats._equity_curve['Equity']

plt.figure(figsize=(10, 6))
plt.plot(equity_curve, label='Equity Curve', lw=1)  # lw is line width
plt.title('Equity Curve')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(False)
plt.show()


In [ ]:

equity_curve = stats._equity_curve['Equity'].reset_index()

fig = px.line(custom_df, x='entry_time', y='account_equity', labels={'index': 'Time'}, title='Strategy Performance')
fig.update_layout(height=600, xaxis_title='Time', yaxis_title='Equity', legend_title='Legend')
fig.show()
